In [ ]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu

In [ ]:
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata

In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key
pc = Pinecone()
pc.create_index(
    name="rag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Asad Hanif',
  'review': 'Very knowledgeable and approachable.',
  'subject': 'Mobile Appliaction Development',
  'stars': 5},
 {'professor': 'Kaleem Ullah',
  'review': 'Clear lectures but strict grading.',
  'subject': 'Web Programming Languages',
  'stars': 4},
 {'professor': 'Shubhan Londhe',
  'review': 'Interesting class, though assignments are tough.',
  'subject': 'DevOps',
  'stars': 5},
 {'professor': 'Muhammad Murtaza',
  'review': 'Very engaging and helpful during office hours.',
  'subject': 'Computer Organization and Assembly Language',
  'stars': 5},
 {'professor': 'Babar Zahoor',
  'review': 'Content is challenging but rewarding.',
  'subject': 'Cloud Computing',
  'stars': 4},
 {'professor': 'Irfan Malik',
  'review': 'Detailed explanations and great support material.',
  'subject': 'Artificial Intelligence',
  'stars': 5},
 {'professor': 'Mukhtar Qureshi',
  'review': 'Difficult exams and bad grading.',
  'subject': 'Database Management Systems',
  'st

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    return hf_embeddings.embed_query(text)

with open('reviews.json', 'r') as file:
    data = json.load(file)

processed_data = []

for review in data['reviews']:
    embedding = get_embedding(review["review"])

    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

print(processed_data)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[{'values': [0.021948985755443573, 0.0023717323783785105, -0.06047692522406578, 0.0065617989748716354, -0.07945440709590912, 0.026172643527388573, 0.030201241374015808, 0.0337664857506752, -0.05028022080659866, 0.05593869462609291, -0.043905213475227356, 0.07413165271282196, 0.018743176013231277, 0.0656617283821106, -0.017268622294068336, -0.045802902430295944, 0.01977868378162384, -0.08652357757091522, -0.00281774764880538, -0.06258287280797958, -0.009816673584282398, 0.0381692573428154, 0.04548069089651108, -0.00967507716268301, -0.040187448263168335, 0.056198373436927795, -0.020912496373057365, -0.027888940647244453, 0.08892413228750229, -0.05173933878540993, -0.003724063979461789, 0.033017996698617935, 0.048086341470479965, 0.043468985706567764, -0.009811104275286198, 0.07917671650648117, 0.017409548163414, 0.07159154117107391, 0.005577004514634609, -0.006014265585690737, -0.07048019766807556, -0.01623007096350193, -0.0026798902545124292, -0.04046514257788658, -0.0190800279378891, 

In [ ]:
processed_data[0]

{'values': [0.021948985755443573,
  0.0023717323783785105,
  -0.06047692522406578,
  0.0065617989748716354,
  -0.07945440709590912,
  0.026172643527388573,
  0.030201241374015808,
  0.0337664857506752,
  -0.05028022080659866,
  0.05593869462609291,
  -0.043905213475227356,
  0.07413165271282196,
  0.018743176013231277,
  0.0656617283821106,
  -0.017268622294068336,
  -0.045802902430295944,
  0.01977868378162384,
  -0.08652357757091522,
  -0.00281774764880538,
  -0.06258287280797958,
  -0.009816673584282398,
  0.0381692573428154,
  0.04548069089651108,
  -0.00967507716268301,
  -0.040187448263168335,
  0.056198373436927795,
  -0.020912496373057365,
  -0.027888940647244453,
  0.08892413228750229,
  -0.05173933878540993,
  -0.003724063979461789,
  0.033017996698617935,
  0.048086341470479965,
  0.043468985706567764,
  -0.009811104275286198,
  0.07917671650648117,
  0.017409548163414,
  0.07159154117107391,
  0.005577004514634609,
  -0.006014265585690737,
  -0.07048019766807556,
  -0.01623

In [ ]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="reviews-1"
)

{'upserted_count': 10}

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}